In [1]:
import jax
import optax
import time

import jax.numpy as jnp
import jax.random as jrd
import numpy as np

from jax.random import PRNGKey
from flax import linen as nn

import sys
sys.path.insert(1, '..')
from kronojax.neural.KAN import KAN, MLP
from kronojax.neural.embedding import time_embedding

In [2]:
# Set random Key
key = jrd.PRNGKey(0)

#### (Unnormalised) Target distribution

In [3]:
def log_target(x):
    """ Mixture of two Gaussians """
    mu1 = -1.
    mu2 = 1.
    sigma2 = 0.09
    proba_1 = jnp.exp(-0.5*(x-mu1)**2 / sigma2)
    proba_2 = jnp.exp(-0.5*(x-mu2)**2 / sigma2)
    return jnp.log(0.5 * proba_1 + 0.5 * proba_2)

#### Parametrize flow and drift functions with neural networks

In [4]:
class NN_drift(nn.Module):
    """parametrize drift function"""

    time_embedding_dim: int
    time_freq_min: float
    time_freq_max: float
    dim_list: list

    @nn.compact
    def __call__(self, x, t):

        t_embedded = time_embedding(t, self.time_freq_min, self.time_freq_max, self.time_embedding_dim)
        input_forward = jnp.concatenate([x, t_embedded], axis=1)
        NN_forward = KAN(dim_list=self.dim_list, degree=2)

        return NN_forward(input_forward)

class NN_flow(nn.Module):
    """parametrize flow function"""

    time_embedding_dim: int
    time_freq_min: float
    time_freq_max: float
    dim_list: list

    @nn.compact
    def __call__(self, x, t):

        t_embedded = time_embedding(t, self.time_freq_min, self.time_freq_max, self.time_embedding_dim)
        input_flow = jnp.concatenate([x, t_embedded], axis=1)
        NN_flow = MLP(dim_list=self.dim_list)

        return NN_flow(input_flow)
    
class NN_FlowAndDirft(nn.Module):
    """concatenate two neural networks for drift and flow"""

    time_embedding_dim: int
    time_freq_min: float
    time_freq_max: float
    dim_list_drift: list
    dim_list_flow: list

    @nn.compact
    def __call__(self, x, t):

        drift = NN_drift(time_embedding_dim=self.time_embedding_dim, 
                         time_freq_min=self.time_freq_min, time_freq_max=self.time_freq_max, dim_list=self.dim_list_drift)
        
        flow = NN_flow(time_embedding_dim=self.time_embedding_dim, 
                       time_freq_min=self.time_freq_min, time_freq_max=self.time_freq_max, dim_list=self.dim_list_flow)

        return drift(x, t), flow(x, t)

In [5]:
key, subkey = jrd.split(key)
time_embedding_dim = 16
time_freq_min = 1.
time_freq_max = 10
output_dim = 1
dim_list_drift = [32, 32, output_dim]
dim_list_flow = [32, 32, output_dim]
GFnet = NN_FlowAndDirft(time_embedding_dim, time_freq_min, time_freq_max, dim_list_drift, dim_list_flow)

# initialize the network
batch_sz = 32
key, subkey = jrd.split(key)
xs = jrd.normal(subkey, (batch_sz, 1))
ts = jrd.uniform(subkey, (batch_sz, 1))
params = GFnet.init(subkey, xs, ts)
params

{'params': {'NN_drift_0': {'KAN_0': {'ChebyKANLayer_0': {'cheby_coeffs': Array([[[-0.07041111,  0.20284824, -0.06126592],
             [ 0.00940971,  0.04172569, -0.1579823 ],
             [-0.02896337,  0.05601507, -0.11375765],
             ...,
             [ 0.1374556 , -0.22028808,  0.2900462 ],
             [-0.1632105 , -0.06120371,  0.04129769],
             [-0.02348674, -0.14249957,  0.25576115]],
     
            [[-0.39078516, -0.13679978,  0.04843318],
             [-0.03109715, -0.00769316, -0.09090478],
             [-0.02482637,  0.18937069,  0.02661465],
             ...,
             [-0.20854132, -0.02448861,  0.16933846],
             [-0.08682724, -0.07608379, -0.11177863],
             [-0.16323403,  0.13063551, -0.0215685 ]],
     
            [[ 0.40600494, -0.12630777, -0.00693014],
             [-0.1319363 ,  0.03731949, -0.09223805],
             [-0.01511926, -0.16144781,  0.24434397],
             ...,
             [ 0.05203369,  0.01608879,  0.14095777],


In [6]:
# print(params_drift['params']['KAN_0']['ChebyKANLayer_0']['cheby_coeffs'].shape)
# print(params_drift['params']['KAN_0']['LayerNorm_0']['scale'].shape)
# print(params_drift['params']['KAN_0']['LayerNorm_0']['bias'].shape)

# print(params_drift['params']['KAN_0']['ChebyKANLayer_1']['cheby_coeffs'].shape)
# print(params_drift['params']['KAN_0']['LayerNorm_1']['scale'].shape)
# print(params_drift['params']['KAN_0']['LayerNorm_1']['bias'].shape)

# print(params_drift['params']['KAN_0']['ChebyKANLayer_2']['cheby_coeffs'].shape)

# print(params_flow['params']['MLP_0']['Dense_0']['kernel'].shape)
# print(params_flow['params']['MLP_0']['Dense_0']['bias'].shape)

# print(params_flow['params']['MLP_0']['Dense_1']['kernel'].shape)
# print(params_flow['params']['MLP_0']['Dense_1']['bias'].shape)

# print(params_flow['params']['MLP_0']['Dense_2']['kernel'].shape)
# print(params_flow['params']['MLP_0']['Dense_2']['bias'].shape)

#### Define the loss function

In [7]:
def log_normal_density(x, mu, sigma2):
    """
    x: n-dim array
    mu: n-dim array
    sigma2: scalar
    """
    return - 0.5 * jnp.log(2 * jnp.pi * sigma2) - 0.5 * (x - mu)**2 / sigma2

In [8]:
def Traj(
        params: any,
        batch_sz: int,
        N_step: int,
        key: PRNGKey
        ):
    """keep track on flow function, forward probability, and backward probability"""

    T = 1.
    dt = T / N_step
    sqt = jnp.sqrt(dt)

    def _step(carry, _):
        xo, t, count, key = carry
        coeff = count / (count + 1)
        to = jnp.ones_like(xo) * t
        u, f = GFnet.apply(params, xo, to)
        key, subkey = jrd.split(key)
        dw = sqt * jrd.normal(subkey, xo.shape)
        xn = xo + u*dt + dw
        t += dt
        count += 1
        log_PF = log_normal_density(xn, xo + u*dt, dt) # Forward probability
        log_PB = jnp.where(
            coeff == 0,
            log_normal_density(xo, coeff * xn, 0.5 * dt), # prevent zero variance
            log_normal_density(xo, coeff * xn, coeff * dt) # Backward probability
            )
        output_dict = {
            "t": t,
            "x": xo,
            "log(P_forward)": log_PF,
            "log(P_backward)": log_PB,
            "state flow": f
            }
        return (xn, t, count, subkey), output_dict
    
    key, subkey = jrd.split(key)
    t_init = 0.
    x_init = jnp.zeros((batch_sz, 1))
    count_init = 0
    carry_init = (x_init, t_init, count_init, subkey)
    _, trajectory = jax.lax.scan(_step, carry_init, xs = None, length=N_step)
    return trajectory

In [9]:
trajectory = Traj(params, batch_sz, 10, key)
print("t: ", trajectory["t"].shape)
print("x: ", trajectory["x"].shape)
print("forward: ", trajectory["log(P_forward)"].shape)
print("backward: ", trajectory["log(P_backward)"].shape)
print("flow: ", trajectory["state flow"].shape)

t:  (10,)
x:  (10, 32, 1)
forward:  (10, 32, 1)
backward:  (10, 32, 1)
flow:  (10, 32, 1)


In [10]:
def loss(
        params, 
        batch_sz: int, 
        N_step: int, 
        key: PRNGKey
        ):
    """loss function -- total trajectory balance"""
    
    trajectory = Traj(params, batch_sz, N_step, key)
    F0 = trajectory["state flow"][0]
    xN = trajectory["state flow"][-1]
    log_uN = log_target(xN)
    ratio1 = jnp.sum(trajectory["log(P_forward)"] - trajectory["log(P_backward)"], axis=0)
    ratio2 = jnp.log(F0) - log_uN + ratio1
    loss = jnp.mean(ratio2**2, axis=0)
    return loss.reshape(())

loss = jax.jit(loss, static_argnums=(1, 2,)) 
loss_value_grad = jax.value_and_grad(loss, argnums=0)

In [11]:
loss(params, 32, 10, key)

Array(1.3569624, dtype=float32)

#### Update parameters via loss function

In [12]:
lr = 1e-3
optimizer = optax.adam(learning_rate=lr)

opt_state = optimizer.init(params)

def update(params, opt_state, batch_sz, N_step, key):
    """update the parameters"""

    loss_value, grads = loss_value_grad(params, N_step, batch_sz, key)
    updates, opt_state = optimizer.update(grads, opt_state)
    new_params = optax.apply_updates(params, updates)
    return new_params, opt_state, loss_value

update = jax.jit(update, static_argnums=(2, 3))

In [13]:
Niter = 10
Batch_SZ = 256
LR = 10**-2
optimizer = optax.adam(learning_rate=LR)

loss_values = []

time_start = time.time()
for i in range(Niter):
    key, subkey = jrd.split(key)
    params, opt_state, loss_value = update(params, opt_state, Batch_SZ, 10, subkey)
    loss_values.append(loss_value)
    print(f"Iteration {i}, Loss {loss_value}")
    if i % 10 == 0:
        time_current = time.time()
        print(f"Iteration {i:4d}/{Niter}  |  Loss: {loss_value:.2f}  |  Time: {time_current - time_start:.2f} s")

Iteration 0, Loss 7.3596930503845215
Iteration    0/10  |  Loss: 7.36  |  Time: 2.61 s
Iteration 1, Loss nan
Iteration 2, Loss nan
Iteration 3, Loss nan
Iteration 4, Loss nan
Iteration 5, Loss nan
Iteration 6, Loss nan
Iteration 7, Loss nan
Iteration 8, Loss nan
Iteration 9, Loss nan
